# 4.2 – Osterdatum und bewegliche Feiertage nach Gauß

Carl Friedrich Gauß veröffentlichte seine so genannte Osterformel erstmals im Jahre 1800. Aufgrund der säkularen Mondschaltung brachte Gauß 1816 eine Korrektur an die Öffentlichkeit, der Gültigkeitsraum ist: 1700-4099.
Der korrigierte Algorithmus von 1816 für den Gregorianischen Kalender für das Jahr x lautet wie folgt: 

```
a = Rest von x / 19 
b = Rest von x / 4 
c = Rest von x / 7 
k = x / 100 
p =  (8*k + 13) / 25 
q = k / 4 
M = Rest von (15 + k -p - q) / 30 
N = Rest von (4 + k - q) / 7 
d = Rest von (19*a + M) / 30 
e = Rest von (2*b + 4*c + 6*d + N) / 7 
Ostern = 22 + d +e 
```

**Ausnahmen:**
  1. Falls `d + e = 35`, dann `Ostern = 50`
  2. Falls `d = 28` und `e = 6` und Rest von `(11*M + 11) / 30 < 19`, dann `Ostern = 49`. Dabei ist Ostern das Datum des Ostersonntags als Tag im März gezählt. Da der März immer 31 Tage hat, ist `Ostern > 31`, ansonsten muss man 31 von dem Ergebnis abziehen und man erhält das Datum im April.

 Erarbeiten Sie ein Python-Programm nach dem vorgegebenen Algorithmus, das für ein einzugebendes Jahr Ostern, Karfreitag, Himmelfahrt, Pfingsten und Fastnacht berechnet. Gliedern Sie dazu die Berechnung von Ostern sowie den zu berechnenden beweglichen Feiertagen in die Funktionen.

### Funktionale Definition der Gaußschen Osterformel mittels Lambda-Funktion

In [ ]:
def osterformel(x):
    return (
        # Lambda-Funktion zur Berechnung der Osterformel
        lambda a, b, c, k, p, q, M, N, d, e: (
            50 if d + e == 35 else
            49 if (d == 28 and e == 6 and ((11 * M + 11) % 30) < 19) else
            22 + d + e # Regulärer Fall
        )
    )(
        # Definition der Parameterwerte, mit denen die Lambdafunktion aufgerufen werden soll
        x % 19,  # a
        x % 4,   # b
        x % 7,   # c
        x // 100,  # k
        (8 * (x // 100) + 13) // 25,  # p
        (x // 100) // 4,  # q
        (15 + (x // 100) - ((8 * (x // 100) + 13) // 25) - ((x // 100) // 4)) % 30,  # M
        (4 + (x // 100) - ((x // 100) // 4)) % 7,  # N
        (19 * (x % 19) + (15 + (x // 100) - ((8 * (x // 100) + 13) // 25) - ((x // 100) // 4)) % 30) % 30,  # d
        (2 * (x % 4) + 4 * (x % 7) + 6 * ((19 * (x % 19) + (15 + (x // 100) - ((8 * (x // 100) + 13) // 25) - ((x // 100) // 4)) % 30) % 30) + (4 + (x // 100) - ((x // 100) // 4)) % 7) % 7  # e
    )

### Funktionale Bestimmung des Osterdatums mithilfe der Gaußschen Osterformel

In [ ]:
def osterdatum(x):
    if osterformel(x) > 31: return osterformel(x) - 31, 4
    else: return osterformel(x), 3

### Funktionale Bestimmung der beweglichen Feiertage basierend auf dem Osterdatum

In [ ]:
def karfreitag(x):
    return (
        lambda ostersonntag_tag, ostersonntag_monat: (
            (ostersonntag_tag - 2, ostersonntag_monat) if ostersonntag_tag > 2
            else (31 + ostersonntag_tag - 2, ostersonntag_monat - 1)
        )
    )(*osterdatum(x))

def fastnacht(x):
    return (
        lambda ostersonntag_tag, ostersonntag_monat: (
            (ostersonntag_tag + 31 - 47, ostersonntag_monat - 1) if ostersonntag_tag + 31 - 47 > 0
            else (31 + ostersonntag_tag + 31 - 47, ostersonntag_monat - 2)
        )
    )(*osterdatum(x))

def himmelfahrt(x):
    return (
        lambda ostersonntag_tag, ostersonntag_monat: (
            (ostersonntag_tag + 40, ostersonntag_monat)
            if ostersonntag_tag + 40 <= 31
            else (
                (ostersonntag_tag + 40 - 31, ostersonntag_monat + 1)
                if ostersonntag_tag + 40 - 31 <= 31
                else (ostersonntag_tag + 40 - 31 - 31, ostersonntag_monat + 2)
            )
        )
    )(*osterdatum(x))

def pfingsten(x):
    return (
        lambda ostersonntag_tag, ostersonntag_monat: (
            (ostersonntag_tag + 49 - 61, ostersonntag_monat + 2) if ostersonntag_tag + 49 > 61
            else (ostersonntag_tag + 49 - 31, ostersonntag_monat + 1) 
        )
    )(*osterdatum(x))

### Testloop – tabellarische Ausgabe der Feiertage der Jahre 2020 – 2030

In [ ]:
def format_datum(datum):
    return f"{datum[0]}.{datum[1]}."

def tabellarische_ausgabe():
    return "\n".join(
        map(
            lambda jahr: f"{jahr}\t{format_datum(fastnacht(jahr))}\t{format_datum(karfreitag(jahr))}\t{format_datum(osterdatum(jahr))}\t{format_datum(himmelfahrt(jahr))}\t{format_datum(pfingsten(jahr))}",
            range(2020, 2031)
        )
    )

print(f"Jahr\tFastn.\tKarfr.\tOstern\tHim.\tPfingsten\n{ tabellarische_ausgabe() }")